# Create Dataset from S3

Install pre-req software


In [4]:
!pip install sagemaker --upgrade
!pip install "transformers>=4.4.2" 
!pip install s3fs --upgrade
!pip install datasets --upgrade 
#!pip install "librosa==0.9.1librosa"
!pip install torch # framework is required for transformer 
!pip install torchaudio

!conda install -y -c conda-forge librosa

Keyring is skipped due to an exception: 'keyring.backends'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.8/629.8 kB 6.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Created wheel for sagemaker: filename=sagemaker-2.122.0-py2.py3-none-any.whl size=854911 sha256=40a82051598885035e7e72a42fec79d68cadcbd3765d1b3d382f1253f8aac946
  Stored in directory: /root/.cache/pip/wheels/db/b7/1b/8cec9d4e83ea0aabf775ba133b50abcdc257a9f466acc1a7fd
Successfully built sagemaker
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.120.0
    Uninstalling sagemaker-2.120.0:
      Successfully uninstalled sagemaker-2.120.0
ERROR: pip's dependency resolver does not currently take into account all the p

In [27]:
!pip3 install torch --no-cache-dir
!pip3 install torchaudio --no-cache-dir
#import librosa

  Using cached torchaudio-0.13.0-cp37-cp37m-manylinux1_x86_64.whl (4.2 MB)


In [26]:
!pip list

Package                              Version
------------------------------------ -----------------
aiobotocore                          2.4.1
aiohttp                              3.8.3
aioitertools                         0.11.0
aiosignal                            1.3.1
alabaster                            0.7.12
anaconda-client                      1.7.2
anaconda-project                     0.8.3
ansi2html                            1.8.0
anyio                                3.6.2
argh                                 0.26.2
argon2-cffi                          21.3.0
argon2-cffi-bindings                 21.2.0
asn1crypto                           1.3.0
astroid                              2.12.13
astropy                              4.0
async-timeout                        4.0.2
asynctest                            0.13.0
atomicwrites                         1.3.0
attrs                                22.1.0
autopep8                             1.4.4
autovizwidget                    

In [5]:
!conda install -c conda-forge libsndfile -y
!pip install soundfile

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - libsndfile


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    gettext-0.21.1             |       h27087fc_0         4.1 MB  conda-forge
    lame-3.100                 |    h166bdaf_1003         496 KB  conda-forge
    libflac-1.4.2              |       h27087fc_0         411 KB  conda-fo

In [17]:
!conda install -c conda-forge librosa -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.

Set up S3 connection

In [6]:
import json
import time
import boto3
import numpy as np
import random
#import soundfile 
import sagemaker
import sagemaker.huggingface

BUCKET="radio-dataset" # please use your bucket name
PREFIX = "bbc" 
ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=BUCKET)

print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::121713061542:role/service-role/AmazonSageMaker-ExecutionRole-20220927T193257
sagemaker bucket: radio-dataset
sagemaker session region: us-west-2


Log in to huggingface

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
from datasets import load_dataset, load_metric, Audio, DatasetDict,load_from_disk

from datasets.filesystems import S3FileSystem
s3 = S3FileSystem()


In [9]:
datasetPath="s3://radio-dataset/bbc/clips/"
dataset = load_dataset("audiofolder", data_dir=datasetPath, filesystem=s3)

EmptyDatasetError: The directory at /root/longonot/sagemaker/prep_datasets/s3:/radio-dataset/bbc/clips doesn't contain any data files

Test that dataset loaded

In [10]:
s3.ls(datasetPath)[:5]

['radio-dataset/bbc/clips/',
 'radio-dataset/bbc/clips/4_11_21-amka_0.wav',
 'radio-dataset/bbc/clips/4_11_21-amka_1.wav',
 'radio-dataset/bbc/clips/4_11_21-amka_10_0_0.wav',
 'radio-dataset/bbc/clips/4_11_21-amka_10_0_1.wav']

In [18]:
import torchaudio
#import librosa

from datasets import load_dataset, load_metric, Audio, DatasetDict,Dataset
sampling_rate = 16000

allFiles = s3.ls(datasetPath)
def gen():
    for item in allFiles:
        if(item.endswith(".wav")):
             with s3.open(item, 'rb') as f:
                arr, sr = torchaudio.load(BytesIO(f), sr=sampling_rate)
                yield item,arr

dataset2 = Dataset.from_generator(gen)

ModuleNotFoundError: No module named 'torchaudio'